In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w3-p2/train_data.csv
/kaggle/input/2021-ai-w3-p2/test_data.csv
/kaggle/input/2021-ai-w3-p2/sample_submit.csv


In [2]:
import torch
import torch.optim as optim
from sklearn import preprocessing

In [3]:
torch.manual_seed(1)

In [4]:
train_data=pd.read_csv("/kaggle/input/2021-ai-w3-p2/train_data.csv")
test_data=pd.read_csv("/kaggle/input/2021-ai-w3-p2/test_data.csv")
submit_data=pd.read_csv("/kaggle/input/2021-ai-w3-p2/sample_submit.csv")
print(train_data.head())
print(test_data.head())
print(submit_data.head())

   ID  company    model  year  transmission  mileage  fueltype  tax   mpg  \
0   0        2    Focus  2019             1     3764         4  145  34.5   
1   1        1      I10  2015             1    27056         4   20  60.1   
2   2        3       A3  2019             3     3375         4  145  34.5   
3   3        3       A6  2019             0     5743         0  145  47.9   
4   4        2   Fiesta  2015             1    32782         4    0  65.7   

   engineSize  price  
0         2.3  26299  
1         1.0   5495  
2         2.0  32400  
3         2.0  29990  
4         1.0   7250  
   ID  company      model  year  transmission  mileage  fueltype  tax   mpg  \
0   0        2        Ka+  2017             1    10150         4  145  57.7   
1   1        2       Kuga  2013             1    50216         0  145  53.3   
2   2        3         Q5  2016             1    34189         0  200  47.9   
3   3        1        I10  2020             1     1900         4  145  56.5   
4   

In [5]:
#preprocessing
le=preprocessing.LabelEncoder()
le.fit(train_data['model'])
train_data['model'] = le.transform(train_data['model'])
test_data['model'] = le.transform(test_data['model'])
print(le.classes_)

[' 1 Series' ' 2 Series' ' 3 Series' ' 4 Series' ' 5 Series' ' 6 Series'
 ' 7 Series' ' 8 Series' ' A1' ' A3' ' A4' ' A5' ' A6' ' A7' ' A8'
 ' Auris' ' Avensis' ' Aygo' ' B-MAX' ' C-HR' ' C-MAX' ' Camry' ' Corolla'
 ' EcoSport' ' Edge' ' Fiesta' ' Focus' ' Fusion' ' GT86' ' Galaxy'
 ' Getz' ' Grand C-MAX' ' Grand Tourneo Connect' ' Hilux' ' I10' ' I20'
 ' I30' ' I40' ' I800' ' IQ' ' IX20' ' IX35' ' Ioniq' ' KA' ' Ka+' ' Kona'
 ' Kuga' ' Land Cruiser' ' M2' ' M3' ' M4' ' M5' ' M6' ' Mondeo'
 ' Mustang' ' PROACE VERSO' ' Prius' ' Puma' ' Q2' ' Q3' ' Q5' ' Q7' ' Q8'
 ' R8' ' RAV4' ' RS3' ' RS4' ' RS5' ' RS6' ' S-MAX' ' S3' ' S4' ' S5'
 ' S8' ' SQ5' ' SQ7' ' Santa Fe' ' Supra' ' TT' ' Tourneo Connect'
 ' Tourneo Custom' ' Tucson' ' Urban Cruiser' ' Veloster' ' Verso'
 ' Verso-S' ' X1' ' X2' ' X3' ' X4' ' X5' ' X6' ' X7' ' Yaris' ' Z3' ' Z4'
 ' i3' ' i8']


In [6]:
x1train = torch.FloatTensor(train_data['company']).cuda()
x2train = torch.FloatTensor(train_data['model']).cuda()
x3train = torch.FloatTensor(train_data['year']).cuda()
x4train = torch.FloatTensor(train_data['transmission']).cuda()
x5train = torch.FloatTensor(train_data['mileage']).cuda()
x6train = torch.FloatTensor(train_data['fueltype']).cuda()
x7train = torch.FloatTensor(train_data['tax']).cuda()
x8train = torch.FloatTensor(train_data['mpg']).cuda()
x9train = torch.FloatTensor(train_data['engineSize']).cuda()
ytrain = torch.FloatTensor(train_data['price']).cuda()

In [7]:
#inital
w1 = torch.zeros(1, requires_grad=True, device = "cuda")
w2 = torch.zeros(1, requires_grad=True,device = "cuda")
w3 = torch.zeros(1, requires_grad=True,device = "cuda")
w4 = torch.zeros(1,requires_grad= True,device = "cuda")
w5 = torch.zeros(1, requires_grad=True,device = "cuda")
w6 = torch.zeros(1, requires_grad=True,device = "cuda")
w7 = torch.zeros(1, requires_grad=True,device = "cuda")
w8 = torch.zeros(1,requires_grad= True,device = "cuda")
w9 = torch.zeros(1,requires_grad= True,device = "cuda")
b = torch.zeros(1, requires_grad=True,device = "cuda")

In [8]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3,w4,w5,w6,w7,w8,w9, b], lr=9.7e-10)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):
    hypothesis = x1train * w1 + x2train * w2 + x3train * w3 + x4train * w4 + x5train * w5+ x6train * w6+x7train * w7+x8train * w8+x9train * w9 + b
    cost = torch.mean((hypothesis - ytrain) ** 2)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print('Epoch {:4d}/{}  Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000  Cost: 383204160.000000
Epoch 1000/10000  Cost: 82995728.000000
Epoch 2000/10000  Cost: 82722824.000000
Epoch 3000/10000  Cost: 82620040.000000
Epoch 4000/10000  Cost: 82518928.000000
Epoch 5000/10000  Cost: 82419360.000000
Epoch 6000/10000  Cost: 82321288.000000
Epoch 7000/10000  Cost: 82224696.000000
Epoch 8000/10000  Cost: 82129576.000000
Epoch 9000/10000  Cost: 82035896.000000
Epoch 10000/10000  Cost: 81943616.000000


In [9]:
x1test = torch.FloatTensor(test_data['company']).cuda()
x2test = torch.FloatTensor(test_data['model']).cuda()
x3test = torch.FloatTensor(test_data['year']).cuda()
x4test = torch.FloatTensor(test_data['transmission']).cuda()
x5test = torch.FloatTensor(test_data['mileage']).cuda()
x6test = torch.FloatTensor(test_data['fueltype']).cuda()
x7test = torch.FloatTensor(test_data['tax']).cuda()
x8test = torch.FloatTensor(test_data['mpg']).cuda()
x9test = torch.FloatTensor(test_data['engineSize']).cuda()
ytest = x1test * w1 + x2test * w2 + x3test * w3 + x4test * w4 + x5test * w5+ x6test * w6+x7test * w7+x8test * w8+x9test * w9 + b
print(ytest)

tensor([19529.7383, 11866.7910, 15160.9873,  ..., 12575.8721, 18666.4023,
         6409.1162], device='cuda:0', grad_fn=<AddBackward0>)


In [10]:
submit_data['price']=ytest.cpu().detach().numpy()
print(submit_data.head())

   ID         price
0   0  19529.738281
1   1  11866.791016
2   2  15160.987305
3   3  21123.083984
4   4   5748.901855


In [11]:
submit_data.to_csv("submission.csv",mode='w',index=False)